# Segmenting and Clustering Neighborhoods in Toronto
## Shuren Qu
## 10/18/2020

### 1. Add Libraries:

#### 1.1 install Geopy

In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: done

# All requested packages already installed.



#### 1.2 install Follium

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



#### 1.3 install BeautifulSoup

In [4]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

# All requested packages already installed.



In [5]:
!conda install -c conda-forge html5lib --yes

Solving environment: done

# All requested packages already installed.



In [6]:
!conda install -c conda-forge lxml --yes

Solving environment: done

# All requested packages already installed.



#### 1.4 Import Libraries

In [7]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import folium
import json
import folium
import html5lib
import lxml
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline 
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs
print('All Libraries Imported')

All Libraries Imported


### 2. Scraping Wikipedia:

#### 2.1 Scraping with BeautifulSoup

In [8]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html, 'html.parser')
trs=soup.find_all('table')[0]

#### 2.2 Make DataFrame

In [9]:
#read from html
df=pd.read_html(str(trs))
df=df[0]

#normolize dataframe
#df['Neighbourhood'].replace("Not assigned", df['Borough'], inplace = True)
df['Neighbourhood'].replace("Not assigned", np.nan, inplace = True)
df.replace("Not assigned", np.nan, inplace = True)
df.dropna(inplace = True)
listhelp=df.values.tolist()
i=0
while i<len(listhelp):
    k=i+1
    while k<len(listhelp):
        if (listhelp[i][0]==listhelp[k][0]):
            listhelp[i][2]=str(listhelp[i][2])+', '+str(listhelp[k][2])
            listhelp[k][0]='del'
        k+=1
    i+=1
newlist=[]
for i in listhelp:
    if i[0]!='del':
        newlist.append(i)

#make dataframe for postcode
df_Toronto=pd.DataFrame(newlist,columns=['Postcode','Borough','Neighbourhood'])
df_Toronto.head(20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### 2.3 Numbers of cells in dataframe

In [10]:
print(df_Toronto.shape)

(103, 3)


### 3. Dataframe with geotag

In [11]:
#df1 geotag
df1 = pd.read_csv('http://cocl.us/Geospatial_data')
dfff=df_Toronto.copy()